In [171]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import librosa
import librosa.display as ld
import IPython
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten
tf.config.run_functions_eagerly(True)
import numpy as np

In [172]:
def load_wav_16k_mono(filename):
    audio_data, sample_rate = librosa.load(filename.numpy().decode('utf-8'), sr=None, mono=True)
    #audio_data, sample_rate = librosa.load(filename, sr=None, mono=True)
    audio_data = librosa.resample(audio_data, orig_sr=sample_rate, target_sr=16000)
    return tf.convert_to_tensor(audio_data, dtype=tf.float32)

@tf.function
def preprocess(file_path: str, label: tf.float32):
    wav = tf.py_function(load_wav_16k_mono, [file_path], [tf.float32])
    # Why 320 and 32?
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    print(f"Spec shape: {spectrogram.shape}")
    spectrogram = tf.abs(spectrogram)
    print(f"Spec shape 2: {spectrogram.shape}")
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    print(f"Spec shape 3: {spectrogram.shape}")
    return spectrogram[0], label
    

# Plot Examples

In [163]:
# QUEEN_EXAMPLE = "./queenless/queen/Hive1_12_06_2018_QueenBee_H1_audio___15_00_00_0.wav"
# NO_QUEEN_EXAMPLE = "./queenless/no_queen/CJ001 - Missing Queen - Day -  (102)_16.wav"
# wave = load_wav_16k_mono(QUEEN_EXAMPLE)
# nwave = load_wav_16k_mono(NO_QUEEN_EXAMPLE)
# print(len(wave), len(nwave))
# spectrogram = tf.signal.stft(wave, frame_length=320, frame_step=32)
# print(f"Spec shape: {spectrogram.shape}")
# spectrogram = tf.abs(spectrogram)
# print(f"Spec shape 2: {spectrogram.shape}")
# spectrogram = tf.expand_dims(spectrogram, axis=2)
# print(f"Spec shape 3: {spectrogram.shape}")
# spectrogram.shape
# print(f"Spec shape 4: {spectrogram[0].shape}")

AttributeError: 'str' object has no attribute 'numpy'

In [89]:
# plt.plot(wave)
# plt.plot(nwave)
# display(IPython.display.Audio(wave, rate = 16000))
# plt.show()

In [173]:
Q = tf.data.Dataset.list_files("./queenless/queen/*.wav")
NQ = tf.data.Dataset.list_files("./queenless/no_queen/*.wav")
Q.as_numpy_iterator().next()

b'./queenless/queen/Hive1_12_06_2018_QueenBee_H1_audio___16_20_00_19.wav'

In [130]:
# sp, l = preprocess(Q.shuffle(100).as_numpy_iterator().next(), None)
# plt.figure(figsize=(30, 20))
# plt.imshow(tf.transpose(sp)[0])
# plt.show()

In [174]:
queens = tf.data.Dataset.zip((Q, tf.data.Dataset.from_tensor_slices(tf.ones(len(Q)))))
no_queens = tf.data.Dataset.zip((NQ, tf.data.Dataset.from_tensor_slices(tf.ones(len(NQ)))))
data = queens.concatenate(no_queens)
print(data.take(1).as_numpy_iterator().next())
e = data.take(1)

(b'./queenless/queen/Hive1_12_06_2018_QueenBee_H1_audio___15_20_00_10.wav', 1.0)


In [177]:
i = e.map(preprocess)
ii = i.as_numpy_iterator().next()
ii = np.array(ii[0])
ii.shape

Spec shape: <unknown>
Spec shape 2: <unknown>
Spec shape 3: <unknown>


(1991, 1, 257)

In [160]:


data = data.map(preprocess)
samples, labels = data.as_numpy_iterator().next()
print(f"Shapee: {samples.shape}")
data = data.cache()
data = data.shuffle(1000)
data = data.batch(16)
data = data.prefetch(8)
len(data), len(data) * 0.7
data

Spec shape: <unknown>
Spec shape 2: <unknown>
Spec shape 3: <unknown>
Shapee: (1991, 1, 257)


<_PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [143]:
train = data.take(43)
test = data.take(19)
samples, labels = train.as_numpy_iterator().next()
samples.shape, labels.shape

2024-02-07 22:12:32.248019: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:413: Filling up shuffle buffer (this may take a while): 451 of 1000
2024-02-07 22:12:44.086348: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


((16, 1991, 1, 257), (16,))

In [144]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(1991, 257, 1)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('Adam', loss='BinaryCrossentropy', metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [145]:
hist = model.fit(train, epochs=4, validation_data=test)

Epoch 1/4


ValueError: Input 0 of layer "sequential_1" is incompatible with the layer: expected shape=(None, 1991, 257, 1), found shape=(16, 1991, 1, 257)